In [52]:
import pandas as pd
import geopandas as gpd
from libpysal.weights import Queen
from spreg import ML_Lag
import numpy as np
import matplotlib.pyplot as plt
from libpysal.weights import lag_spatial
from spreg import GM_Lag
import plotly.express as px
import json

In [53]:
crimes = pd.read_csv('data/all_crimes_2022-2025.csv')
crimes['Month'] = pd.to_datetime(crimes['Month'])

In [54]:
crimes_cleaned = crimes.dropna(subset=['Longitude', 'Latitude'])

burglary = crimes_cleaned[crimes_cleaned['Crime type'] == 'Burglary'].copy()

print(f"Original dataset: {len(crimes)} rows")
print(f"After dropping missing coords: {len(crimes_cleaned)} rows")
print(f"Number of burglaries: {len(burglary)}")

Original dataset: 3386817 rows
After dropping missing coords: 3340179 rows
Number of burglaries: 158454


In [55]:
LSOAs = gpd.read_file('data/LSOAs.geojson').to_crs(epsg=4326)

gdf_crimes = gpd.GeoDataFrame(
    crimes_cleaned,
    geometry=gpd.points_from_xy(crimes_cleaned['Longitude'], crimes_cleaned['Latitude']),
    crs="EPSG:4326"
)

gdf_crimes= gpd.sjoin(
    gdf_crimes,
    LSOAs[['geometry', 'LSOA11NM']],
    how='left',
    predicate='within'
)

gdf_burglary = gpd.GeoDataFrame(
    burglary,
    geometry=gpd.points_from_xy(burglary['Longitude'], burglary['Latitude']),
    crs="EPSG:4326"
)

gdf_burglary = gpd.sjoin(
    gdf_burglary,
    LSOAs[['geometry', 'LSOA11NM']],
    how='left',
    predicate='within'
)

monthly_burglary_counts = (
    gdf_burglary.dropna(subset=['LSOA11NM'])
    .groupby(['LSOA11NM', gdf_burglary['Month'].dt.to_period('M')])
    .size()
    .reset_index(name='Burglary_Count')
)

monthly_burglary_counts['Month'] = monthly_burglary_counts['Month'].dt.to_timestamp()

In [56]:
num_lsoas = len(LSOAs)
print(f"Number of LSOAs in the file: {num_lsoas}")

num_unique_lsoas_crimes = gdf_crimes['LSOA11NM'].nunique()
print(f"Number of unique LSOAs in crimes dataset: {num_unique_lsoas_crimes}")

num_unique_lsoas_burglary = gdf_burglary['LSOA11NM'].nunique()
print(f"Number of unique LSOAs in burglary dataset: {num_unique_lsoas_burglary}")

Number of LSOAs in the file: 4835
Number of unique LSOAs in crimes dataset: 4834
Number of unique LSOAs in burglary dataset: 4829


In [57]:
# Fill months with no burglaries with 0
burglary_pivot = monthly_burglary_counts.pivot(index='LSOA11NM', columns='Month', values='Burglary_Count').fillna(0)

full_lsoa_months = pd.MultiIndex.from_product(
    [monthly_burglary_counts['LSOA11NM'].unique(), monthly_burglary_counts['Month'].unique()],
    names=['LSOA11NM', 'Month']
)

monthly_burglary_counts = monthly_burglary_counts.set_index(['LSOA11NM', 'Month']).reindex(full_lsoa_months, fill_value=0).reset_index()
monthly_burglary_counts

# burglary_pivot = monthly_burglary_counts.pivot(index='LSOA11NM', columns='Month', values='Burglary_Count').fillna(0)

,LSOA11NM,Month,Burglary_Count
0,Barking and Dagenham 001A,2022-03-01,4
1,Barking and Dagenham 001A,2022-04-01,1
2,Barking and Dagenham 001A,2022-07-01,2
3,Barking and Dagenham 001A,2022-10-01,1
4,Barking and Dagenham 001A,2023-03-01,1
...,...,...,...
173839,Westminster 024F,2023-05-01,0
173840,Westminster 024F,2023-10-01,1
173841,Westminster 024F,2024-08-01,1
173842,Westminster 024F,2025-02-01,0


In [58]:
missing_lsoas = list(set(LSOAs['LSOA11NM']) - set(burglary_pivot.reset_index()['LSOA11NM']))
print(missing_lsoas)

['Newham 015E', 'City of London 001A', 'Haringey 013D', 'Bromley 027E', 'Croydon 030B', 'Tower Hamlets 031F']


In [59]:
missing_rows = pd.DataFrame(
    0,
    index=missing_lsoas,
    columns=burglary_pivot.columns
)

burglary_pivot = pd.concat([burglary_pivot, missing_rows])

burglary_pivot = burglary_pivot.sort_index()

In [60]:
print(type(burglary_pivot.columns))
burglary_pivot

<class 'pandas.core.indexes.datetimes.DatetimeIndex'>


Month,2022-03-01,2022-04-01,2022-05-01,2022-06-01,2022-07-01,2022-08-01,2022-09-01,2022-10-01,2022-11-01,2022-12-01,...,2024-05-01,2024-06-01,2024-07-01,2024-08-01,2024-09-01,2024-10-01,2024-11-01,2024-12-01,2025-01-01,2025-02-01
Barking and Dagenham 001A,4.0,1.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,...,2.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,2.0,0.0
Barking and Dagenham 001B,0.0,0.0,0.0,1.0,0.0,4.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
Barking and Dagenham 001C,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0
Barking and Dagenham 001D,0.0,1.0,2.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,2.0,0.0,0.0,2.0,0.0,1.0,1.0,0.0
Barking and Dagenham 002A,1.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Westminster 024B,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,1.0,0.0,1.0,1.0,1.0,4.0,2.0,3.0,0.0
Westminster 024C,1.0,0.0,1.0,1.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0
Westminster 024D,2.0,1.0,1.0,0.0,0.0,2.0,3.0,1.0,2.0,2.0,...,1.0,4.0,0.0,0.0,1.0,1.0,3.0,3.0,2.0,0.0
Westminster 024E,1.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0


In [61]:
# Sort months for encoding on time
months_sorted = sorted(burglary_pivot.columns)

month_mapping = {m: i for i, m in enumerate(months_sorted)}
burglary_pivot.columns = [month_mapping[m] for m in burglary_pivot.columns]
burglary_pivot

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
Barking and Dagenham 001A,4.0,1.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,...,2.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,2.0,0.0
Barking and Dagenham 001B,0.0,0.0,0.0,1.0,0.0,4.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
Barking and Dagenham 001C,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0
Barking and Dagenham 001D,0.0,1.0,2.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,2.0,0.0,0.0,2.0,0.0,1.0,1.0,0.0
Barking and Dagenham 002A,1.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Westminster 024B,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,1.0,0.0,1.0,1.0,1.0,4.0,2.0,3.0,0.0
Westminster 024C,1.0,0.0,1.0,1.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0
Westminster 024D,2.0,1.0,1.0,0.0,0.0,2.0,3.0,1.0,2.0,2.0,...,1.0,4.0,0.0,0.0,1.0,1.0,3.0,3.0,2.0,0.0
Westminster 024E,1.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0


In [62]:
# Make sure that LSOAs_filtered contains only LSOAs also in burglary_pivot
# lsoas_in_burglary = burglary_pivot.index.tolist()
# LSOAs_filtered = LSOAs[LSOAs['LSOA11NM'].isin(lsoas_in_burglary)].copy()

# This will make total number of LSOAs evaluated smaller, thus falsely decreases number of neighbors for an LSOA, which makes weight of remaining LSOAs higher

In [63]:
# Make sure LSOAs_filtered contains only LSOAs also in Jan 2025
# (This is not the reason we do this at the end, but it does still work lol, since it makes sure the order of rows is alphabetically instead of on E01000001 numbers, so it now matches with burglary_pivot)
lsoas_jan_2025 = burglary_pivot.iloc[:, -2].index
print(len(lsoas_jan_2025))

LSOAs_filtered = LSOAs[LSOAs['LSOA11NM'].isin(lsoas_jan_2025)].copy()
LSOAs_filtered = LSOAs_filtered.set_index('LSOA11NM').loc[lsoas_jan_2025].reset_index()
# print(len(LSOAs_filtered))

4835


In [64]:
# Spatial weights using LSOA boundaries
# w = Queen.from_dataframe(LSOAs, use_index=False)
w = Queen.from_dataframe(LSOAs_filtered, use_index=False)
w.transform = 'r'

In [65]:
# Check out neighbors
neighbors_df = pd.DataFrame([
    {'LSOA': k, 'Neighbors': v} for k, v in w.neighbors.items()
])

neighbors_df

,LSOA,Neighbors
0,0,"[1, 2289, 3, 4, 3733, 3687, 2318]"
1,1,"[0, 2, 3, 3733]"
2,2,"[1, 3, 4, 5, 3733, 3757, 3758]"
3,3,"[0, 1, 2, 4]"
4,4,"[0, 2, 3, 5, 6, 8, 2318, 2351, 2320]"
...,...,...
4830,4830,"[4832, 4833, 2926, 4818, 4821, 4822, 4825, 482..."
4831,4831,"[4832, 4817, 4834, 4815]"
4832,4832,"[4833, 4818, 4834, 4830, 4831]"
4833,4833,"[4832, 4830]"


In [66]:
# X1, X2, X3 independent variables: Dec, Nov, Oct 2024
X_dec = burglary_pivot.iloc[:, -3].values.reshape(-1, 1)
X_nov = burglary_pivot.iloc[:, -4].values.reshape(-1, 1)
X_oct = burglary_pivot.iloc[:, -5].values.reshape(-1, 1)

# Y is Jan 2025 burglary rates
Y = burglary_pivot.iloc[:, -2].values.reshape(-1, 1)

In [67]:
# Check if dimensions match
w_matrix, _ = w.full()
print("Shape of spatial weights matrix w:", w_matrix.shape)
print("Shape of Y.flatten():", Y.flatten().shape)

Shape of spatial weights matrix w: (4835, 4835)
Shape of Y.flatten(): (4835,)


In [68]:
# Spatial lag of Y (computed automatically by ML_Lag)
WY = lag_spatial(w, Y.flatten()).reshape(-1, 1)

# Combine independent variables: Dec, Nov, Oct 2024 in this case
X_combined = np.hstack([X_dec, X_nov, X_oct])

# Train SAR model
model = ML_Lag(Y, X_combined, w=w, name_y="Burglary_Jan", name_x=["Dec", "Nov", "Oct"])

# Predict burglary for next month (Feb 2025), based on SAR predictions of Jan 2025 (should be comparable)
predicted_burglary = model.predy

In [ ]:
# Plot predictions
predicted_df = pd.DataFrame({
    'LSOA11NM': burglary_pivot.index,
    'Predicted_Burglary': predicted_burglary.flatten()
})

predicted_gdf = LSOAs.merge(predicted_df, on='LSOA11NM', how='left')

fig = px.choropleth_map(
    predicted_gdf,
    geojson=json.loads(LSOAs.to_json()),
    locations='LSOA11NM',
    featureidkey="properties.LSOA11NM",
    color='Predicted_Burglary',
    color_continuous_scale="YlOrRd",
    range_color=(0, predicted_df['Predicted_Burglary'].max()),
    map_style="open-street-map",
    zoom=9,
    center={"lat": 51.5072, "lon": -0.1276},
    opacity=0.7,
    height=600
)
fig.update_layout(title="Predicted Burglary Rate (Feb 2025), based on SAR predictions of Jan 2025")
fig.show()

In [ ]:
# Plot actual rates
burglary_jan_2025 = monthly_burglary_counts[
    monthly_burglary_counts['Month'] == pd.Timestamp("2025-01-01")
]

missing_rows = pd.DataFrame({
    'LSOA11NM': missing_lsoas,
    'Month': [pd.Timestamp("2025-01-01")] * len(missing_lsoas),
    'Burglary_Count': [0] * len(missing_lsoas)
})

burglary_feb_2025 = pd.concat([burglary_jan_2025, missing_rows])
burglary_feb_2025 = burglary_feb_2025.sort_values(by=['LSOA11NM'])

jan_map_gdf = LSOAs.merge(
    burglary_jan_2025,
    on="LSOA11NM",
    how="left"
)

fig = px.choropleth_map(
    jan_map_gdf,
    geojson=json.loads(LSOAs.to_json()),
    locations='LSOA11NM',
    featureidkey="properties.LSOA11NM",
    color='Burglary_Count',
    color_continuous_scale="YlOrRd",
    range_color=(0, burglary_feb_2025['Burglary_Count'].max()),
    map_style="open-street-map",
    zoom=9,
    center={"lat": 51.5072, "lon": -0.1276},
    opacity=0.7,
    height=600
)

fig.update_layout(title="Actual Burglary Rate (Jan 2025)")
fig.show()

In [ ]:
# Plot actual rates
burglary_feb_2025 = monthly_burglary_counts[
    monthly_burglary_counts['Month'] == pd.Timestamp("2025-02-01")
]

missing_rows = pd.DataFrame({
    'LSOA11NM': missing_lsoas,
    'Month': [pd.Timestamp("2025-02-01")] * len(missing_lsoas),
    'Burglary_Count': [0] * len(missing_lsoas)
})

burglary_feb_2025 = pd.concat([burglary_feb_2025, missing_rows])
burglary_feb_2025 = burglary_feb_2025.sort_values(by=['LSOA11NM'])

feb_map_gdf = LSOAs.merge(
    burglary_feb_2025,
    on="LSOA11NM",
    how="left"
)

fig = px.choropleth_map(
    feb_map_gdf,
    geojson=json.loads(LSOAs.to_json()),
    locations='LSOA11NM',
    featureidkey="properties.LSOA11NM",
    color='Burglary_Count',
    color_continuous_scale="YlOrRd",
    range_color=(0, burglary_feb_2025['Burglary_Count'].max()),
    map_style="open-street-map",
    zoom=9,
    center={"lat": 51.5072, "lon": -0.1276},
    opacity=0.7,
    height=600
)

fig.update_layout(title="Actual Burglary Rate (Feb 2025)")
fig.show()

In [72]:
# Print predicted burglaries per LSOA for Feb 2025 (actually Jan 2025)
predicted_df_sorted = predicted_df.sort_values(by='Predicted_Burglary', ascending=False)

print(predicted_df_sorted)

              LSOA11NM  Predicted_Burglary
4773  Westminster 013E           12.266417
3567       Newham 013G           10.735782
4771  Westminster 013B           10.395247
4774  Westminster 013F            7.787374
969        Camden 028D            6.777952
...                ...                 ...
779       Bromley 030C            0.186556
3826    Redbridge 035E            0.186556
2263       Harrow 033H            0.186556
570         Brent 021C            0.186556
415        Bexley 019F            0.186556

[4835 rows x 2 columns]


In [73]:
# Print actual burglaries per LSOA for Jan 2025
top_lsoas_jan = burglary_jan_2025.sort_values(by='Burglary_Count', ascending=False)

print(top_lsoas_jan)

                LSOA11NM      Month  Burglary_Count
171627  Westminster 013E 2025-01-01              19
66795       Hackney 027G 2025-01-01              16
171555  Westminster 013B 2025-01-01              15
171591  Westminster 013D 2025-01-01              14
34827        Camden 028C 2025-01-01              13
...                  ...        ...             ...
79467        Harrow 021B 2025-01-01               0
79539        Harrow 021D 2025-01-01               0
79575        Harrow 022A 2025-01-01               0
79647        Harrow 022C 2025-01-01               0
173823  Westminster 024F 2025-01-01               0

[4829 rows x 3 columns]


In [74]:
# Print actual burglaries per LSOA for Feb 2025
top_lsoas_feb = burglary_feb_2025.sort_values(by='Burglary_Count', ascending=False)

print(top_lsoas_feb)

                LSOA11NM      Month  Burglary_Count
128302       Newham 013G 2025-02-01              15
171646  Westminster 013E 2025-02-01              12
39670       Croydon 027C 2025-02-01              12
171214  Westminster 011B 2025-02-01              12
115774      Lambeth 036E 2025-02-01               9
...                  ...        ...             ...
76246      Haringey 036B 2025-02-01               0
76318      Haringey 036D 2025-02-01               0
76462      Haringey 037D 2025-02-01               0
76534        Harrow 001B 2025-02-01               0
173842  Westminster 024F 2025-02-01               0

[4835 rows x 3 columns]


In [75]:
burglary_dec_2024 = monthly_burglary_counts[
    monthly_burglary_counts['Month'] == pd.Timestamp("2024-12-01")
]

top_lsoas_dec = burglary_dec_2024.sort_values(by='Burglary_Count', ascending=False)

burglary_nov_2024 = monthly_burglary_counts[
    monthly_burglary_counts['Month'] == pd.Timestamp("2024-11-01")
]

top_lsoas_nov = burglary_nov_2024.sort_values(by='Burglary_Count', ascending=False)

burglary_oct_2024 = monthly_burglary_counts[
    monthly_burglary_counts['Month'] == pd.Timestamp("2024-10-01")
]

top_lsoas_oct = burglary_oct_2024.sort_values(by='Burglary_Count', ascending=False)

In [76]:
# print(top_lsoas_dec)

In [77]:
# print(top_lsoas_nov)

In [78]:
# print(top_lsoas_oct)